In [32]:
import codecs
import requests
import re
import datetime
from bs4 import BeautifulSoup

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

class Ppomppu(object):
    def __init__(self, date_from, date_to, keyword, start_page):
        self.date_from = date_from
        self.date_to = date_to
        self.keyword = keyword
        self.start_page = start_page
    
    # on Board
    def crawl(self):
        f = codecs.open('ppomppu_{}.txt'.format(self.keyword), 'a', 'utf-8')
        page_num = self.start_page
        num_contents = 0
        
        while True:
            hdr = {'User-Agent' : 'Mozilla/5.0',
                   'referer' : 'https://ppomppu.co.kr/'}
            url = "http://www.ppomppu.co.kr/search_bbs.php?search_type=subject&page_no={0}&keyword={1}&page_size=20&bbs_id=&order_type=date&bbs_cate=2".format(page_num, self.keyword)
            res = requests.get(url, headers = hdr)
            soup = BeautifulSoup(res.content, 'html5')
            links = soup.find_all('span', attrs={'class': 'title'})
            
            date_soup = soup.find_all('p', attrs={'class': 'desc'})
            
            try:
                raw_date = re.search(r'\d{4}.\d{2}', date_soup[1].get_text()).group()
                chk_date = raw_date.replace('.', '')
            except AttributeError:
                raw_date = re.search(r'\d{4}.\d{2}', date_soup[2].get_text()).group()
                chk_date = raw_date.replace('.', '')

            print(page_num)
            page_num += 1

            if int(chk_date) <= int(self.date_to) and int(chk_date) > int(self.date_from):
                for link in links:
                    try:
                        link = link.a['href']
                        res2 = requests.get(link, headers = hdr)
                        soup2 = BeautifulSoup(res2.content, 'html5')
                        contents = soup2.find_all('div', attrs={'class': 'container'})
                        num_contents += 1
                        print('processing... ' + str(num_contents) + ' ' + str(link))

                    
                        for form in contents:
                            try:
                                container = form.select('td.han')[1].get_text()
                                date = re.search(r'\d{4}-\d{2}-\d{2}', container).group()
                                view = re.search(r'\d+\s/', container).group()[0:-2]
                                title = form.select('font.view_title2')[0].get_text()
                                content = re.sub(r'\s+', ' ', form.select('td.board-contents')[0].get_text())
                                time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

                                f.write(link+'\t'+date+'\t'+view+'\t'+title+'\t'+content+'\t'+time+'\n')
                            except IndexError:
                                pass
                    except TypeError:
                        pass
                    
            elif int(chk_date) == int(self.date_from):
                print('Reached to date limit ('+str(self.date_to)+', yyyy-mm)')
                break
            else:
                pass
        f.close()
        print("Contents crawling has been completed, total " + str(num_contents) + ' contents')


In [33]:
f.close()

NameError: name 'f' is not defined

In [34]:
#p_g5 = Ppomppu(201512, 201607, 'g5', 566)
#p_g5.crawl()

p_g6 = Ppomppu(201703, 201705, 'g6', 1)
p_g6.crawl()

#p_s6 = Ppomppu(201412, 201507, 's6', 1)
#p_s6.crawl()

#p_s7 = Ppomppu(201512, 201607, 's7', 42)
#p_s7.crawl()